In [1]:
import csv
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import dates
import datetime
from datetime import timedelta

In [2]:
def import_files(file_path, data_type, airport, date):

    filename = '{0}/{1}/{2}/{1}_{3}.txt'.format(file_path, data_type, airport, date) 

    if not os.path.isfile(filename):
        return -1

    data = pd.read_csv(filename)

    data['TIME'] = data['TIME'].astype(str) # convert time to string so '--' can be replaced

    data['TIME'] = data['TIME'].str.replace('--', '00') # replace null seconds with 00   

    data['TIME'] = pd.to_datetime(data['TIME'], format='%Y%m%d%H%M%S')

    data.set_index('TIME', inplace=True)

    return(data)

In [5]:
def main():

    #---------------------------------------------------------------------
    # 01. Settings
    #---------------------------------------------------------------------

    file_path = 'D:/MetDB/'
    out_path = 'D:/Mode-S_altitude/Mode-S_day_min'

    period = 'winter' # summer or winter

    if period == 'summer':
        start_date = datetime.date(2021,7,10)
        #end_date = datetime.date(2021,7,15)
        end_date = datetime.date(2021,8,10)

    if period == 'winter':
        start_date = datetime.date(2022,1,19)
        end_date = datetime.date(2022,1,24)

    date_list = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]
    date_list = [date_obj.strftime('%Y%m%d') for date_obj in date_list]

    airport_name_list = ['Aberdeen']
    '''
    airport_name_list = ['Heathrow', \
                     'Gatwick', \
                     'Manchester', \
                     'Stansted', \
                     'Edinburgh', \
                     'Birmingham', \
                     'Bristol', \
                     'Glasgow', \
                     'Aberdeen', \
                     'EastMidlands', \
                     'LondonCity', \
                     'BelfastInt', \
                     'Newcastle', \
                     'LeedsBradford', \
                     'Liverpool',\
                     'Cardiff']
    '''
    #---------------------------------------------------------------------    
    # 02. Loop through airports/dates and plot altitude
    #--------------------------------------------------------------------- 
    ptGeoms = []
    for airport in airport_name_list:

        concat_amdar = pd.DataFrame()
        concat_modes = pd.DataFrame()
        concat_modes_day_min = pd.DataFrame()

        for date in date_list:

            print('Loading data for: {0} {1}'.format(airport, date))

            data_amdar = import_files(file_path, 'AMDARS', airport, date)
            data_modes = import_files(file_path, 'MODE-S', airport, date)
            
            #Find row containing daily min
            data_modes_day_min = data_modes[data_modes.GNSS_ALTD==data_modes.GNSS_ALTD.min()]

            concat_amdar = pd.concat([concat_amdar, data_amdar])
            concat_modes = pd.concat([concat_modes, data_modes])
            
            concat_modes_day_min = pd.concat([concat_modes_day_min, data_modes_day_min])
 
        # Create dataframe of daily minimum Mode-S altitudes 
        concat_modes_day_min.reset_index(inplace=True)         
        concat_modes_day_min['TIME'] = concat_modes_day_min['TIME'].dt.strftime('%Y%m%d')    
        concat_modes_day_min = concat_modes_day_min.drop_duplicates(['TIME'], keep='first')
        #concat_modes_day_min.to_csv(os.path.join(out_path, "{0}_{1}_LL.csv".format(airport, period)))
        
        
        # Convert day min lat/lon to British Grid - SLOW WAY TO DO IT
        gt = 'OSGB_1936_To_WGS_1984_7'  
        inputSRS = arcpy.SpatialReference(4326) # GCS WGS84    
        outputSRS = arcpy.SpatialReference(7405) # British National Grid 
        #ptGeoms = []
        
        for idx, row in concat_modes_day_min.iterrows():
            pt = arcpy.Point() # create point object 
            pt.Y = row['LAT']
            pt.X = row['LON']
            print("Input XY: {0} {1}".format(pt.X, pt.Y))
            ptGeoms.append(arcpy.PointGeometry(pt,inputSRS ))
            arcpy.CopyFeatures_management(ptGeoms, 'base')
            arcpy.Project_management('base',gt, outputSRS)
            
        #with arcpy.da.SearchCursor("WGS_1984_To_OSGB_1936",['SHAPE@X','SHAPE@Y']) as cursor:
            #for row in cursor:
                #print(row[0])
                #print(row[1])
        '''   
        # Convert day min lat/lon to British Grid 
        gt = 'OSGB_1936_To_WGS_1984_7'  
        inputSRS = arcpy.SpatialReference(4326) # GCS WGS84    
        outputSRS = arcpy.SpatialReference(7405) # British National Grid       
        
        for idx, row in concat_modes_day_min.iterrows():
            pt = arcpy.Point() # create point object 
            pt.Y = row['LAT']
            pt.X = row['LON']
            #print("Input XY: {0} {1}".format(pt.X, pt.Y))
            ptgeo = arcpy.PointGeometry(pt, inputSRS)
            ptgeo1 = ptgeo.projectAs(outputSRS, gt)
            pt1 = ptgeo1.lastPoint
            #print("Output XY: {0} {1}".format(pt1.X, pt1.Y))
            new_lon = round(pt1.X,2)
            new_lat = round(pt1.Y,2)
            concat_modes_day_min.loc[idx,'LAT'] = new_lat
            concat_modes_day_min.loc[idx,'LON'] = new_lon
        
        print(concat_modes_day_min)
        concat_modes_day_min.to_csv(os.path.join(out_path, "{0}_{1}_day_min_BG.csv".format(airport, period)),index=False)
        '''

In [6]:
main()

Loading data for: Aberdeen 20220119
Loading data for: Aberdeen 20220120
Loading data for: Aberdeen 20220121
Loading data for: Aberdeen 20220122
Loading data for: Aberdeen 20220123
Loading data for: Aberdeen 20220124
Input XY: -2.19272 57.19492
Input XY: -2.18877 57.18997
Input XY: -2.18791 57.18901
Input XY: -2.20636 57.21336
Input XY: -1.96612 57.11311
Input XY: -2.21117 57.21991
